In [3]:
import json
import urllib
import requests
import seaborn as sns
import pandas as pd
import datetime
from matplotlib import pyplot as plt
import plotly.graph_objs as go
import plotly.plotly as py
from climata.acis import StationDataIO
%matplotlib inline

In [6]:
class Gage(object): 
    BASE_URL = "http://waterservices.usgs.gov/nwis/iv/?"
    def __init__(self, gageid):
        self.gageid = gageid
        self.queryParams = {
            'site' : self.gageid,
            'format' : 'json',
        }
    
    def _exec_query(self,params):
        url = self.BASE_URL+urllib.parse.urlencode(params)
        df = pd.read_json(json.dumps(requests.get(url).json()['value']['timeSeries'][0]['values'][0]['value']))    
        return df.set_index(pd.to_datetime(df["dateTime"]))
        
    def get_param(self, param=None, startTime=None, endTime=None, period=None):
        thisQuery = self.queryParams.copy()
        thisQuery['parameterCd'] = param
        if startTime and endTime:
            thisQuery['startDT'] = startTime
            thisQuery['endDT']   = endTime
        elif period:
            thisQuery['period'] = period

        
        return self._exec_query(thisQuery)

In [38]:
g = Gage(12178000)
flowdata = g.get_param("00060", startTime = datetime.datetime(2017, 1,15).isoformat(), endTime=datetime.datetime.now().isoformat())
#gagedata = g.get_param("63680", startTime = datetime.datetime(2017, 1,15).isoformat(), endTime=datetime.datetime.now().isoformat())

dam = Gage("12175000")
height = dam.get_param("62614", startTime = datetime.datetime(2017, 1,15).isoformat(), endTime=datetime.datetime.now().isoformat())


In [39]:
wx_station = "Hozomeen Camp"
wxdata = StationDataIO(basin="17110005", start_date='2017-01-01', end_date="2017-03-20", parameter='pcpn').as_dataframe()
precip = wxdata[wxdata['name'] == wx_station]["data"].values[0].as_dataframe()
precip

,date,pcpn
0,2017-01-01,0
1,2017-01-02,0.1
2,2017-01-03,0.1
3,2017-01-04,0
4,2017-01-05,0
5,2017-01-06,0
6,2017-01-07,0
7,2017-01-08,0
8,2017-01-09,0.4
9,2017-01-10,0


In [40]:
flow = go.Scatter(x=flowdata["dateTime"], y=flowdata["value"], yaxis='y1', name="12200500 Discharge")
wx = go.Scatter(x=precip["date"], y=precip["pcpn"], yaxis='y2', name=wx_station+" wx")
ht = go.Scatter(x=height["dateTime"], y=height["value"], yaxis='y3', name="diablo height")
layout = go.Layout(
    title='',
    yaxis=dict(
        title='Discharge (cfs)'
    ),
    yaxis2=dict(
        title='Precip (in)',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ), 
    yaxis3=dict(
    overlaying='y', 
    side='right')
)

fig = go.Figure(data=[flow, wx, ht], layout=layout)
py.iplot(fig, name="testplot")

In [41]:
tarn = Gage("12181090")
tarn.get_param("00060")

,dateTime,qualifiers,value
dateTime,,,
2015-08-16 21:15:00,2015-08-16 21:15:00,[P],47
